# Exercise

You were given a task to display measurement from continuous glucose monitoring device (CGM) `GlucoSpark`. Control enginners from your company have already setup test device that sends data through socket on port `65432` directly on your machine.
The signal send by a device is a comma separated line with `<eventTime>,<glucoseMeasurement>,<displayUnit>,<cgmId>`, that ends with newline sign.
Unfortunetly, device sometimes catches backgroud noise signal and displays irrational, negative glucose measuremets. 
The end user only needs timestamp of the measurement and meaurement value, necessary to detect anomalies in blood test reading.


1. Read streaming data using `socket` format, with host being `127.0.0.1` and port `65432`
2. Split device input signal to seperate columns
3. Cast `eventTime` to *timestamp* type and `glucoseMeasurement` to *integer*
4. Filter negative glucose measurements
5. Select only `eventTime` and `glucoseMeasurement`
6. Write data to console in `append` mode
7. Trigger reading measurements every `1 minute` interval

To start mesurement, go to terminal and run script in `exercises` directory
> python sever_glucose.py

This starts a server to listens to incoming connections and when connection has been made, sends device data through a socket

# TODO

In [9]:
# solution

from pyspark.sql import SparkSession
from pyspark.sql.functions import split

host, port = ("127.0.0.1", "65432")

spark = SparkSession \
    .builder \
    .appName("GlucoSpark") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format('socket') \
    .option('host', host) \
    .option('port', port) \
    .load()

CSVData = lines.select(\
        split(lines.value, ',').getItem(0).alias("eventTime").cast("timestamp"),\
        split(lines.value, ',').getItem(1).alias("glucoseMeasurement").cast("int"),\
        split(lines.value, ',').getItem(2).alias("displayUnit"),\
        split(lines.value, ',').getItem(3).alias("deviceID")
        )

selectAndFilter = CSVData.select("eventTime","glucoseMeasurement")\
        .where("glucoseMeasurement > 0")
    
query = selectAndFilter \
    .writeStream \
    .queryName("BloodTest") \
    .outputMode("append") \
    .format("console") \
    .trigger(processingTime="1 minute") \
    .start()

24/04/21 12:59:44 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/04/21 12:59:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-51bb0ce6-b095-4b0f-8456-7706303aaf12. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/21 12:59:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+------------------+
|eventTime|glucoseMeasurement|
+---------+------------------+
+---------+------------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+------------------+
|           eventTime|glucoseMeasurement|
+--------------------+------------------+
|2024-04-21 12:59:...|               123|
|2024-04-21 12:59:...|               122|
|2024-04-21 12:59:...|               108|
|2024-04-21 12:59:...|               119|
+--------------------+------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+------------------+
|           eventTime|glucoseMeasurement|
+--------------------+------------------+
|2024-04-21 13:00:...|               106|
|2024-04-21 13:00:...|               120|
|2024-04-21 13:00:...|               

24/04/21 13:01:05 WARN TextSocketMicroBatchStream: Stream closed by 127.0.0.1:65432


In [10]:
# Cleanup Classroom
query.stop()
spark.stop()